In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [5]:
import pandas as pd
import datasets
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
config = LongformerConfig(hidden_size=512)
config

In [7]:
# Read in dataset
df = pd.read_csv('training_set.csv')
df = df[['text','label']]


In [ ]:
# Split dataset into train and test sets
train, test = train_test_split(df, test_size=0.2, shuffle=True)
train_data = datasets.Dataset.from_pandas(train, features=datasets.Features({'text':datasets.Value('string'),'label':datasets.Value('int64')}))
test_data = datasets.Dataset.from_pandas(test, features=datasets.Features({'text':datasets.Value('string'),'label':datasets.Value('int64')}))

test.head()

In [ ]:
# Load model and tokenizer
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                           gradient_checkpointing=False,
                                                           attention_window = 512)
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = 1024)


In [ ]:
model.config


In [ ]:
# Tokenize text
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = 'max_length', truncation=True, max_length = 1024)

train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))


In [12]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


In [13]:
# Accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [14]:
# Training arguments
training_args = TrainingArguments(
    learning_rate = 3e-5,
    output_dir = 'results/',
    num_train_epochs = 1,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1,    
    per_device_eval_batch_size= 16,
    evaluation_strategy = "epoch",
    disable_tqdm = False, 
    load_best_model_at_end=True,
    warmup_steps=200,
    weight_decay=0.01,
    logging_steps = 4,
    fp16 = True,
    logging_dir='results/',
    dataloader_num_workers = 0,
    run_name = 'longformer-classification-updated-rtx3090_paper_replication_2_warm'
)

In [ ]:
# Instantiate trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


In [ ]:
# Train the model
trainer.train()


In [18]:
# Save the model
trainer.save_model('model')

In [ ]:
# Evaluate the model on test set
trainer.evaluate()


In [ ]:
# Predict labels for test set
logits, label_ids, metrics = trainer.predict(test_data)
pred_0 = []
pred_1 = []
for pair in logits:
  prediction = torch.nn.functional.softmax(torch.tensor(pair),dim=0)
  pred_0.append(prediction.numpy()[0])
  pred_1.append(prediction.numpy()[1])



In [22]:
# Save predicted labels
d = {'text':test_data['text'],'label':test_data['label'],'predicted prob of being pos':pred_1}
df_results = pd.DataFrame(data=d)
df_results['correct prediction?'] = np.where(abs(df_results['label']-df_results['predicted prob of being pos'])<0.5,'Yes','No')

df_results.to_csv("test_set_results.csv")